# <span style="color:#0073e6">0. 사례준비</span>
<style>
@media print
{
h1 {page-break-before:always}
}
</style>

> **Libaray & Environment Settings** 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense

np.random.seed(1234)

> **CHRUN DATA SET LOADING**

In [2]:
churn = pd.read_csv('data/churn_case_ver01.csv', index_col='customerID')
churn.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,PaymentMethod_B,PaymentMethod_C,PaymentMethod_E,PaymentMethod_M
customerID,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,1,0,1,0,1,0,0,1,0,1,...,0,0,0,29.85,29.85,0,0,0,1,0
5575-GNVDE,0,0,0,0,34,1,0,1,1,0,...,0,1,1,56.95,1889.50,0,0,0,0,1
3668-QPYBK,0,0,0,0,2,1,0,1,1,1,...,0,0,0,53.85,108.15,1,0,0,0,1
7795-CFOCW,0,0,0,0,45,0,0,1,1,0,...,0,1,1,42.30,1840.75,0,1,0,0,0
9237-HQITU,1,0,0,0,2,1,0,2,0,0,...,0,0,0,70.70,151.65,1,0,0,1,0


- - -
# <span style="color:#2EFE9A">.</span>
> **데이터 준비**
> > Tran/Test 분할 및 스케일링 : Min-max

In [3]:
y = churn.Churn
x = churn.drop('Churn', axis='columns')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)
mm = MinMaxScaler()
mm.fit(x_train)
x_train = mm.transform(x_train)
x_test = mm.transform(x_test)

> > Baseline모델 학습

In [4]:
rf = RandomForestClassifier(max_depth=10)
rf.fit(x_train, y_train)
rf_pred = rf.predict_proba(x_test)
roc_auc_score(y_test, rf_pred[:,1])

0.8339007111748167

- - -
# <span style="color:#0073e6">1. AutoEncoder의 적용</span>
### 1.1. 모델 정의 및 학습

In [5]:
num_dim = 10
auto_en = Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(num_dim, activation='relu'),
    Dense(x_train.shape[1], activation='relu')
])
auto_en.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                230       
_________________________________________________________________
dense_1 (Dense)              (None, 22)                242       
Total params: 472
Trainable params: 472
Non-trainable params: 0
_________________________________________________________________


In [6]:
epochs = 100
batch = 128
auto_en.compile(loss='mean_squared_error', optimizer='RMSProp', metrics=['mse'])
auto_en.fit(x_train, x_train, batch_size=batch, epochs=epochs, verbose=0)

- - -
# <span style="color:#2EFE9A">.</span>
### 1.2. 특성 추출 및 적용

In [7]:
active_model = Model(inputs=auto_en.input, outputs=auto_en.layers[0].output)
x_train_ae = active_model.predict(x_train)
x_test_ae = active_model.predict(x_test)
print(x_train_ae.shape)

(4930, 10)


In [8]:
rf = RandomForestClassifier(max_depth=10)
rf.fit(x_train_ae, y_train)
rf_pred = rf.predict_proba(x_test_ae)
roc_auc_score(y_test, rf_pred[:,1])

0.7753557022621607